In [1]:
!pip install psycopg2-binary

  Using cached psycopg2_binary-2.9.5-cp310-cp310-macosx_11_0_arm64.whl (2.0 MB)


In [2]:
import psycopg2
conn = psycopg2.connect(dbname='ideas', user='ideas', 
                        password='ideas2022', host='collabro.ru')
cursor = conn.cursor()